In [ ]:
from starlette.middleware.cors import CORSMiddleware
from fastapi import FastAPI, File, UploadFile, Form

from ultralytics import YOLO
from PIL import Image
import ultralytics
import torch

from typing import List, Tuple
import numpy as np
import datetime
import psycopg2
import random
import uuid
import cv2
import os

from copy import deepcopy

# app = FastAPI()

# ## app.include_router(get_info.router)

# origins = ["*"]

# app.add_middleware(
#     CORSMiddleware,
#     allow_origins=origins,
#     allow_credentials=True,
#     allow_methods=["*"],
#     allow_headers=["*"],
# )

YOLO_IMG_DIM = (640, 640)
DEVICE = "cpu" if not torch.cuda.is_available() else "cuda"
if DEVICE == "cuda":
    torch.cuda.set_device(0)
DEVICE = torch.device(DEVICE)

MODEL_BASE_DIR = "saved_models"
CAR_LABEL_DETECTOR_PATH = os.path.join(
    MODEL_BASE_DIR, "detector_pytorch", "car_detection_yolov9e.pt"
)
CAR_COLOR_CLASSIFIER_PATH_PATH = os.path.join(
    MODEL_BASE_DIR, "detector_pytorch", "color_classification_yolov8x.pt"
)
CAR_LABELS = [
    "bus",
    "car_honda",
    "car_hyundai",
    "car_other",
    "car_suzuki",
    "car_toyota",
    "number_plate",
    "van_honda",
    "van_hyundai",
    "van_other",
    "van_suzuki",
    "van_toyota",
]
NUMBER_PLATE_LABEL = "number_plate"

COLOR_NAMES = ["black", "blue", "green", "grey", "red", "silver", "white"]

db_connection_params = {
    "database": "postgres",
    "user": "postgres",
    "password": "root",
    "host": "localhost",
    "port": "5432",
}

car_label_detector = YOLO(CAR_LABEL_DETECTOR_PATH)
car_color_classifier = YOLO(CAR_COLOR_CLASSIFIER_PATH_PATH)


def detect_car_labels(image: np.ndarray) -> List:
    image = cv2.resize(image, YOLO_IMG_DIM, interpolation=cv2.INTER_LINEAR)
    image = torch.from_numpy(image).to(DEVICE)
    image = torch.permute(image, (2, 0, 1))
    image = torch.unsqueeze(image, axis=0)
    image = image / 255.0
    results = car_label_detector(image)
    return results


def save_image_to_path(image: np.ndarray, path: str) -> None:
    print("Saving image to path: ", path)
    image = Image.fromarray(image)
    image.save(path)


def process_frame(frame: np.ndarray) -> List[ultralytics.engine.results.Results]:
    results = detect_car_labels(frame)
    return results


In [ ]:
PLOT = True
COLORS = np.random.randint(0, 255, size=(len(CAR_LABELS), 3))

img = cv2.imread("test.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = process_frame(img)

for result in results:
    cls = result.boxes.cls
    conf = result.boxes.conf
    xyxy = result.boxes.xyxy
    
    if PLOT:
        for idx, bbox in enumerate(xyxy):
            class_id = int(cls[idx])
            color = COLORS[class_id]
            B, G, R = map(int, color)
            
            x1, y1, x2, y2 = map(int, bbox)
            
            cv2.rectangle(img, (x1, y1), (x2, y2), (B, G, R), 2)

In [17]:
print(cls)
print(conf)
print(xyxy)

tensor([10., 10., 10.,  6., 10.,  4.,  6.,  6.,  6.,  5.])
tensor([0.9310, 0.9235, 0.8725, 0.7845, 0.7711, 0.7255, 0.6816, 0.6640, 0.4419, 0.3875])
tensor([[437.3596, 177.9492, 484.0812, 266.3586],
        [297.5886, 203.9937, 398.1119, 328.8989],
        [222.0328, 284.1886, 311.0915, 500.2911],
        [245.6956, 444.0231, 269.6535, 462.8244],
        [141.3698, 193.1443, 221.4624, 277.5515],
        [225.8426, 210.5190, 281.2885, 288.3285],
        [ 93.4183, 486.5626, 118.4189, 506.1341],
        [237.7191, 273.7083, 251.9913, 282.0500],
        [461.9178, 247.1688, 473.8709, 254.0199],
        [ 79.5081, 343.8776, 223.2977, 567.5811]])


In [ ]:
img2 = deepcopy(img)

In [ ]:
["output_c_1.mp4", "output_c_2.mp4", "output_c_3.mp4"]

ffmpeg -ss 00:00:00 -to 00:02:00 -i output-file1.mp4 -c copy output_c_1.mp4
ffmpeg -ss 00:00:00 -to 00:02:00 -i output-file2.mp4 -c copy output_c_2.mp4
ffmpeg -ss 00:00:00 -to 00:02:00 -i output-file3.mp4 -c copy output_c_3.mp4


ffmpeg -y -i NVR_ch12_main_20231108140011_20231108150000.dav -c:v libx264 -crf 24 output-file1.mp4
ffmpeg -y -i NVR_ch1_main_20231108140000_20231108150000.dav -c:v libx264 -crf 24 output-file2.mp4
ffmpeg -y -i NVR_ch7_main_20231108140006_20231108150000.dav -c:v libx264 -crf 24 output-file3.mp4

In [ ]:
ffmpeg -i processed_output_c_1.avi -vf "setpts=5*PTS" slowed_1.avi
ffmpeg -i processed_output_c_2.avi -vf "setpts=5*PTS" slowed_2.avi
ffmpeg -i processed_output_c_3.avi -vf "setpts=5*PTS" slowed_3.avi